In [79]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import model_function

In [80]:
exponential_count =2
parameter_count = exponential_count*2+1

Load a model or creat a new one?

In [81]:
load = True
model_path ='models/2024-01-08_16-14-55_2exp_params_205_points_50/model.keras'

In [82]:
df = pd.read_csv('measurements/C_4_1000_20231213_14_15_50.csv', sep=', ')
# df = pd.read_csv('measurements/E_4_20ms_20231103_14_55_46.csv', sep=', ')
df.head()

C:\Users\crpdn\AppData\Local\Temp\ipykernel_16416\431649043.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('measurements/C_4_1000_20231213_14_15_50.csv', sep=', ')


,Ch1(mA),Ch2(mA),Ch3(mA),Ch4(mA),Ch5(mA),Ch6(mA),Ch7(mA),Ch8(mA),Ch9(mA),Ch10(mA),...,Ch14(mA),Ch15(mA),Ch16(mA),Channel state,On time,Off time,Pulse number,Pattern,Time,Probeflag
0,0,0,0,0.03421,0,0,0,0,0,0,...,0,0,0,1000000000000,75,22,1,1/1000,0.087664,False
1,0,0,0,0.04888,0,0,0,0,0,0,...,0,0,0,1000000000000,75,22,2,1/1000,0.182584,False
2,0,0,0,0.06354,0,0,0,0,0,0,...,0,0,0,1000000000000,75,22,3,1/1000,0.283189,False
3,0,0,0,0.08309,0,0,0,0,0,0,...,0,0,0,1000000000000,75,22,4,1/1000,0.376060,False
4,0,0,0,0.09775,0,0,0,0,0,0,...,0,0,0,1000000000000,75,22,5,1/1000,0.469858,False


In [83]:
target_points = 50
cutoff_current_min = 0
must_be_growth = False
must_be_decay = False
min_gradient =0
max_final_gradient = 1

In [84]:

number_of_epochs = 1000

In [85]:
not_completed = []
predicted_param_list = []

In [86]:
import keras

import tensorflow.keras.backend as K

def custom_loss(y_true, y_pred):
    """
    Custom loss function for the specified task.

    Parameters:
    - y_true: Ground truth values, of shape (batch_size, 2, N).
    - y_pred: Predicted values, of shape (batch_size, 7,N).

    Returns:
    - loss: Scalar value representing the mean loss over the batch.
    """

    # Extract x and y values from y_true
    x_values = y_true[:,:,0]  # Shape: (batch_size, N)
    y_values = y_true[:,:,1]  # Shape: (batch_size, N)
    
    function_values = tf.zeros_like(x_values)
    for i in range(int((y_pred.shape[2]-1)/2)):
        temp_val= y_pred[:, :,i]*i * K.exp(-(x_values) * y_pred[:, :,i+1]*i)
        function_values = function_values+ temp_val
    function_values = function_values+y_pred[:,:,-1]
    
    # square absolute error
    diff = (y_values -function_values) 
    squared_diff = tf.math.square(diff)
    mse = tf.math.reduce_sum(squared_diff, axis=-1)


    # diff = function_values - y_values
    # # print(diff)
    # # Take the square of the differences
    # squared_diff = K.square(diff)

    # slope_comparison = 0.0
    # for i in range(y_true.shape[2]-1):
    #     x_diff = x_values[:,i+1]-x_values[:,i]  
    #     function_slope = (function_values[:,i+1]-function_values[:,i])/x_diff 
    #     true_slope= (y_values[:,i+1]-y_values[:,i])/x_diff
    #     slope_comparison += tf.math.square(true_slope- function_slope)      

    # square relative error
    # diff = (y_values -function_values) / (y_values+0.0001) #K.maximum(y_values, y_values+0.00001)
    # squared_diff = tf.math.square(diff)
    # mse = tf.math.reduce_sum(squared_diff, axis=-1)

    loss = mse #+ slope_comparison
    # loss = K.mean(squared_diff, axis=-1)
    return loss
 

def create_nn():
    global X_train, points
    # Input layer, the number of input nodes is governed by X_data.shape[1]
    # X_data.shape[1] is the number of columns in X_data
    inputs = keras.Input(shape=(X_train.shape[1],), name='input')
    
    # Dense layers 
    #layers_norm = keras.layers.BatchNormalization()(inputs)
    layers_dense = keras.layers.Dense(10, 'linear')(inputs)
    layers_dense2 = keras.layers.Dense(10, 'linear')(layers_dense)
    # layers_dense3 = keras.layers.Dense(20, 'linear')(layers_dense2)
    # layers_dense4 = keras.layers.Dense(20, 'linear')(layers_dense3)
    # Parameter layer
    # layers_norm = keras.layers.LayerNormalization()(layers_dense2)
    parameters = keras.layers.Dense(parameter_count)(layers_dense2)
    # Expand parameters to have same shape as y_true
    expanded_parameters = keras.layers.RepeatVector(points)(parameters)


    return keras.Model(inputs=inputs, outputs=expanded_parameters, name="current_function_prediction")



def compile_model(model):
    sgd = keras.optimizers.RMSprop(clipnorm=5)
    model.compile(optimizer=sgd, #'adam'
                  loss=custom_loss)
    




earlystopper = keras.callbacks.EarlyStopping(monitor="loss",baseline = 1, patience=number_of_epochs,restore_best_weights=True)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='loss', 
                              factor=0.8, 
                              patience=5, 
                              min_lr=1e-6)


import seaborn as sns
def plot_history(history, metrics, y_lim):
    """
    Plot the training history

    Args:
        history (keras History object that is returned by model.fit())
        metrics (str, list): Metric or a list of metrics to plot
    """
    history_df = pd.DataFrame.from_dict(history.history)
    sns.lineplot(data=history_df[metrics])
    if y_lim !=0:
        plt.ylim(0,y_lim)
    plt.xlabel("epochs")
    plt.ylabel("metric")


In [87]:
df = df[ ['Pattern','Time','Ch4(mA)','On time', 'Off time']]
df = df.rename(columns={'Ch4(mA)': 'Current'})
df_original = df.drop(df.index[:50*10])

for i in  range(10):#range(int(df.shape[0]/50)):
    data = df_original.iloc[i*50:(i+1)*50]

    x_time = data['Time']-np.min(data['Time'])
    y_current = data['Current']
    points = len(y_current) 
    X_data = np.zeros((1,3))
    #initialize array of expected shape
    y_data = np.zeros((1, points, 2))
    
    label = np.column_stack((x_time,y_current))  
    y_data[0,:,:]= label
    features =  np.array([np.min(y_current.values), data['On time'].iloc[0], data['Off time'].iloc[0]])
    X_data[0,:] = features

    X_train = X_data
    y_train = y_data

    if load:
        model = keras.models.load_model(model_path, custom_objects={'custom_loss':custom_loss})
    else:
        model = create_nn()

    compile_model(model)

    history = model.fit(X_train, y_train,
                    batch_size=1,
                    epochs=number_of_epochs,
                    # validation_data = (X_val,y_val),
                    callbacks=[earlystopper
                               ,reduce_lr
                               ],
                    verbose=0)
    history_df = pd.DataFrame.from_dict(history.history)
    final_loss = history_df['loss'].iloc[-1]
    if final_loss < 0.1:
        y_train_prediction = model.predict(X_train)
        param_predicted = y_train_prediction[0,0 ,: ]
        predicted_param_list.append(param_predicted)
        # print(param_predicted)
    
    else:
        # print(i, final_loss)
        not_completed.append(i)

1/1 [==============================] - 0s 43ms/step
[40.47883     0.77834743  0.31371278 59.995045    0.66604257]
1/1 [==============================] - 0s 44ms/step
[24.182735   -0.05506551 19.532312   40.713486    0.6680008 ]
2 4.362328052520752
1/1 [==============================] - 0s 43ms/step
[11.095563   -0.26184753  0.506366   24.168495    0.7301295 ]
4 0.622421145439148
1/1 [==============================] - 0s 42ms/step
[14.793246   0.7258802  0.7371788 22.336782   1.0697961]
1/1 [==============================] - 0s 42ms/step
[48.37644    0.7528579  0.2896145 43.238537   0.2737618]
1/1 [==============================] - 0s 42ms/step
[18.702034   -0.7459009   0.21245039 60.45004     1.1195121 ]
1/1 [==============================] - 0s 53ms/step
[21.222027    0.18979119  2.893267   36.7751      0.8377145 ]
9 4.665775943200014e+18


In [88]:
print(not_completed)
print(predicted_param_list)

[2, 4, 9]
[array([40.47883   ,  0.77834743,  0.31371278, 59.995045  ,  0.66604257],
      dtype=float32), array([24.182735  , -0.05506551, 19.532312  , 40.713486  ,  0.6680008 ],
      dtype=float32), array([11.095563  , -0.26184753,  0.506366  , 24.168495  ,  0.7301295 ],
      dtype=float32), array([14.793246 ,  0.7258802,  0.7371788, 22.336782 ,  1.0697961],
      dtype=float32), array([48.37644  ,  0.7528579,  0.2896145, 43.238537 ,  0.2737618],
      dtype=float32), array([18.702034  , -0.7459009 ,  0.21245039, 60.45004   ,  1.1195121 ],
      dtype=float32), array([21.222027  ,  0.18979119,  2.893267  , 36.7751    ,  0.8377145 ],
      dtype=float32)]
